## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
%matplotlib inline

camera_model = None

In [ ]:
class CameraModel():
    '''
    Class to represent the camera model. Provides function for undistoring image
    '''
    
    def __init__(self, calibration_images):
        '''
        calibration_images - list of paths to calibration images
        '''

        mtx, dist = self._calibrate_camera(calibration_images)
        self._camera_matrix = mtx
        self._distortion = dist
        
    def undistort(self, image):
        '''
        takes an image and returns an undistorted copy
        
        image - the image to undistort
        '''
        return cv2.undistort(image, self._camera_matrix, self._distortion, None, self._camera_matrix)
        
    def _calibrate_camera(self, calibration_images):
        '''
        Computes the camera intrinsics and extrinsics.
        
        calibration_images - list of paths to calibration images        
        '''
        objpoints = [] # 3d points in real world space
        imgpoints = [] # 2d points in image plane

        nx = 9 # num corners on the x axis
        ny = 6 # num corners on the y axis

        # Prepare object points: (0,0,0), (1,0,0), (2,0,0) ... (8,5,0)
        objp = np.zeros((ny * nx, 3), np.float32)
        objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2) # gen x,y coordinates

        for file in calibration_images:
            img = mpimg.imread(file)

            gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

            if ret:
                imgpoints.append(corners)
                objpoints.append(objp)
            else:
                print("Failed to find corners in", file)

        # Now use calibrate camera to get the intrinsics (and ignore extrinsics)
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

        return mtx, dist

cal_images = glob.glob('./camera_cal/calibration*.jpg')

if camera_model is None:
    camera_model = CameraModel(cal_images)    

# Now test the distortion
test_image = mpimg.imread('./camera_cal/calibration1.jpg')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
fig.tight_layout()
ax1.imshow(test_image)
ax1.set_title('Original Image')
ax2.imshow(camera_model.undistort(test_image))
ax1.set_title('Undistorted Image')


In [ ]:
assert(camera_model is not None)

## Create Thresholded Binary Image

In [ ]:
def threshold_image(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    '''
    takes an image and returns a thresholded binary image that best accentuates 
    the lanes in the image. Uses a sobel operation with respect to x on a grayscale 
    version of the image and binary ors it with the a thresholded S-channel from an HLS 
    copy of the image.
    
    img - the image to process
    s_thresh - thesholds for the S channel of the HLS space
    sx_thresh - thresholds for the sobel operation in the x direction
    '''
    img = np.copy(img)
    # Convert to HSV color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    s_channel = hls[:,:,2]
    
    # Sobel x
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255 * abs_sobelx / np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    combined_binary = np.zeros_like(s_channel)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    return combined_binary

test_image_filenames = glob.glob('./test_images/*.jpg')
test_images = [camera_model.undistort(mpimg.imread(img)) for img in test_image_filenames]

results = [threshold_image(img) for img in test_images]

# Plot the result
f, axes = plt.subplots(len(results), 2, figsize=(24, 70))
f.tight_layout()

for i in range(len(test_images)):
    ax1 = axes[i][0]
    ax1.imshow(test_images[i])
    ax1.set_title('Original Image', fontsize=40)

    ax2 = axes[i][1]
    ax2.imshow(results[i], cmap='gray')
    ax2.set_title('Pipeline Result', fontsize=40)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)


## Perspective Transform

In [ ]:
from numpy.linalg import inv

def get_perspective_transform_funcs():
    '''
    Creates and returns two functions for applying a perspective transform and
    reversing it.
    '''
    # src and dst points found externally. Used to compute the perspective
    # transform.
    src = np.float32([[582, 460], [702, 460], [1100, 719], [205, 719]])
    dst = np.float32([[313, 200], [941, 200], [941, 719], [313, 719]])
    
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = inv(M)
    
    def apply_perspective_transform(img):
        '''
        Takes an image and performs a perspective transformation
        '''
        height, width = img.shape[:2]
        return cv2.warpPerspective(img, M, (width, height), flags=cv2.INTER_LINEAR)
    
    def apply_reverse_transform(img):
        '''
        Takes an image and performs a perspective transformation
        '''
        # Warp the blank back to original image space using inverse perspective matrix (Minv)
        return cv2.warpPerspective(img, Minv, (img.shape[1], img.shape[0])) 
    
    return apply_perspective_transform, apply_reverse_transform

# get the transformation functions
transform_perspective, reverse_transform_perspective = get_perspective_transform_funcs()

def load_and_transform_image(filename):
    '''
    takes a file path to an image, loads it, undistorts it, theholds it,
    and then performs a perspective transform on it to give an over-head view.
    
    returns: original image, undistorted image, binary thresholded image, perspective-transformed image
    '''
    orig = mpimg.imread(filename)
    undist = camera_model.undistort(orig)
    binary = threshold_image(undist)
    binary_warped = transform_perspective(binary)
    return orig, undist, binary, binary_warped

orig, undist, binary, binary_warped = load_and_transform_image('./test_images/straight_lines1.jpg')

f, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, figsize=(24, 30))
x = [582, 702, 1100, 205, 582]
y = [460, 460, 719, 719, 460]
left_line_x = 313
right_line_x = 941
f.tight_layout()
ax1.imshow(undist)
ax1.plot(x, y, color='red')
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(binary_warped, cmap="gray")
ax2.set_title('Undistorted and warped', fontsize=50)
ax2.axvline(left_line_x, color='red')
ax2.axvline(right_line_x, color='red')

orig, undist, binary, binary_warped = load_and_transform_image('./test_images/straight_lines2.jpg')
ax3.imshow(undist)
ax3.plot(x, y, color='red')
ax3.set_title('Original Image', fontsize=50)
ax4.imshow(binary_warped, cmap="gray")
ax4.set_title('Undistorted and warped', fontsize=50)
ax4.axvline(left_line_x, color='red')
ax4.axvline(right_line_x, color='red')

orig, undist, binary, binary_warped = load_and_transform_image('./test_images/test1.jpg')
ax5.imshow(undist)
ax5.plot(x, y, color='red')
ax5.set_title('Original Image', fontsize=50)
ax6.imshow(binary_warped, cmap="gray")
ax6.set_title('Undistorted and warped', fontsize=50)
ax6.axvline(left_line_x, color='red')
ax6.axvline(right_line_x, color='red')
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

## Detect and draw lanes

In [ ]:
orig, undist, binary, binary_warped = load_and_transform_image('./test_images/straight_lines1.jpg')
histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
plt.plot(histogram)
plt.imshow(binary_warped, cmap='gray')

In [ ]:
def detect_lines_cached(binary_warped, left_fit, right_fit):
    '''
    Detects lane lines in an image - assumes that you have previously detected
    lane lines, and are providing the polynomials for the previous lines. Uses
    that information to better inform the search in the new image
    
    binary_warped - new image to search for lane line in
    left_fit - polynomial coefficients for the previous left lane line
    right_fit - polynomial coefficients for the previous right lane line
    
    returns: y values for points, x values for left line, x values for right line, 
             left polynomial coefficients, right polynomial coefficients
    '''
    # Assume you now have a new warped binary image 
    # from the next frame of video (also called "binary_warped")
    # It's now much easier to find line pixels!
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return ploty, left_fitx, right_fitx, left_fit, right_fit


def detect_lines(binary_warped, draw=False):
    '''
    Detects lane lines in an image
    
    binary_warped - new image to search for lane line in
    draw - bool - whether or not to plot the lane lines
    
    returns: y values for points, x values for left line, x values for right line, 
         left polynomial coefficients, right polynomial coefficients
    '''
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    #out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    #out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    if draw:
        plt.imshow(out_img)
        plt.plot(left_fitx, ploty, color='yellow')
        plt.plot(right_fitx, ploty, color='yellow')
        plt.xlim(0, 1280)
        plt.ylim(720, 0)
    
    return ploty, left_fitx, right_fitx, left_fit, right_fit


def calc_curvature(img_shape, ploty, left_fitx, right_fitx, left_fit, right_fit):
    '''
    Calculates the curvature of the lines in world space and the deviation from 
    the center of the lane.
    
    Returns: left lane radius of curvature in meters, right lane radius of curvature in meters,
             deviation from the center of the lane in meters.
    '''
    # Now calc curvature
    y_eval = np.max(ploty)
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    img_height = img_shape[0] * ym_per_pix
    img_width = img_shape[1] * xm_per_pix
    
    left = left_fit_cr[0] * img_height ** 2 + left_fit_cr[1] * img_height + left_fit_cr[2]
    right = right_fit_cr[0] * img_height ** 2 + right_fit_cr[1] * img_height + right_fit_cr[2]
    center = (left + right) / 2.0
    
    off_center = (center - img_width / 2.0)
    
    # Now our radius of curvature is in meters
    return left_curverad, right_curverad, off_center
    
    
orig, undist, binary, binary_warped = load_and_transform_image('./test_images/straight_lines1.jpg')
ploty, left_fitx, right_fitx, left_fit, right_fit = detect_lines(binary_warped, draw=True)
print(calc_curvature(undist.shape, ploty, left_fitx, right_fitx, left_fit, right_fit))

In [ ]:
from numpy.linalg import inv

def draw_overlay(undist, ploty, left_fitx, right_fitx, plot=True):
    '''
    draws an overlay onto the provided image highlighting the lane
    
    undist - undistorted image of the road to draw the lane on top of
    ploty - y values for lanes
    left_fitx - x values for left line
    right_fitx - x values for right line
    left_fit - polynomial coefficients for the previous left lane line
    right_fit - polynomial coefficients for the previous right lane line
    
    returns: image with lanes overlay
    '''
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))

    newwarp = reverse_transform_perspective(color_warp)
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    if plot:
        plt.imshow(result)
    return result

_ = draw_overlay(undist, ploty, left_fitx, right_fitx)

## Putting it all together

In [ ]:
left_fit = None
right_fit = None
        
def process_image(orig):
    '''
    Takes an distorted image and returns an undistorted image with lanes overlaid
    '''
    global left_fit, right_fit
    undist = camera_model.undistort(orig)
    binary = threshold_image(undist)
    binary_warped = transform_perspective(binary)
    if left_fit is not None and right_fit is not None:
        ploty, left_fitx, right_fitx, left_fit, right_fit = detect_lines_cached(binary_warped, left_fit, right_fit)
    else:
        ploty, left_fitx, right_fitx, left_fit, right_fit = detect_lines(binary_warped)

    result = draw_overlay(undist, ploty, left_fitx, right_fitx, plot=False)
    
    left_curv, right_curv, off_center = calc_curvature(undist.shape, ploty, left_fitx, right_fitx, left_fit, right_fit)
    curve_text = "Curvature: Left = {:.2f}, Right = {:.2f}".format(left_curv, right_curv) 
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(result, curve_text, (20, 50), font, 1, (0,255,0), 2)
    off_center_text = "Deviation from center = {:.2f} m".format(off_center)   
    cv2.putText(result, off_center_text, (20, 80), font, 1, (0,255,0), 2)
    
    return result

def process_image_file(filename):
    '''
    Takes a path to a distorted image and plots an undistorted image with lanes overlaid
    '''
    orig, undist, binary, binary_warped = load_and_transform_image(filename)
    ploty, left_fitx, right_fitx, left_fit, right_fit = detect_lines(binary_warped)
    left_curv, right_curv, off_center = calc_curvature(undist.shape, ploty, left_fitx, right_fitx, left_fit, right_fit)
    
    curve_text = "Curvature: Left = {:.2f}m, Right = {:.2f}m".format(left_curv, right_curv) 
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(undist, curve_text, (20, 50), font, 1, (0,255,0), 2)
    off_center_text = "Deviation from center = {:.2f}m".format(off_center)   
    cv2.putText(undist, off_center_text, (20, 80), font, 1, (0,255,0), 2)
    
    
    draw_overlay(undist, ploty, left_fitx, right_fitx)
    
    
process_image_file('./test_images/test5.jpg')

## Running on videos

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

output = 'output.mp4'
clip1 = VideoFileClip("project_video.mp4")
clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time clip.write_videofile(output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))